In this notebook, we'll implement the standard Q-learning technique using pure Python.

In [ ]:
from random import random, randint, choice
from tqdm.auto import tqdm
import numpy as np

The function below trains an agent using Q learning given a transition and reward matrix, both with size `(Nr. of States, Nr. of Actions)`, using `None` values for transitions which are not allowed.

In [ ]:
def train_agent(transition_matrix, reward_matrix, 
        valid_initial_states=None, 
        end_states=None,
        gamma=0.8, alpha=1, epsilon=0, episodes=5000):

  nr_states  = len(transition_matrix)
  nr_actions = len(transition_matrix[0])
  Q = [[0.0 for a in range(nr_actions)] for s in range(nr_states)]

  if valid_initial_states is None:
    # By default, all states are valid initial states
    valid_initial_states = range(nr_states)

  if end_states is None:
    # No end states, so we will infer end states from the transition matrix
    end_states = []

  for episode in tqdm(range(episodes)):
    # Select a random initial state
    state = choice(valid_initial_states)

    # While the goal state hasn't been reached
    while state not in end_states:
      possible_actions_in_current_state = [a for a in range(nr_actions) if transition_matrix[state][a] is not None]
      if not len(possible_actions_in_current_state):
        # Nothing else we can do here
        break

      if random() < epsilon:        # Explore: pick an action at random
        action = choice(possible_actions_in_current_state)
      else:                         # Exploit: pick an action based on the Q matrix
        best_Q = max([Q[state][a] for a in possible_actions_in_current_state])
        best_actions = [a for a in possible_actions_in_current_state if Q[state][a] == best_Q]
        action = choice(best_actions)
      
      next_state = transition_matrix[state][action]
      possible_actions_in_next_state = [a for a in range(nr_actions) if transition_matrix[next_state][a] is not None]
      
      if not len(possible_actions_in_next_state):
        # Nothing else we can do there
        break

      # Get maximum Q value for this next state based on all possible actions
      best_Q_in_next_state = max([Q[next_state][a] for a in possible_actions_in_next_state])
      
      # Update Q and set next state as current one
      Q[state][action] = Q[state][action] + alpha * (reward_matrix[state][action] + gamma * best_Q_in_next_state - Q[state][action])

      # And move on the the next state
      state = next_state

  return Q

Let's now use this function based on the example provided. First, we define the topology of our maze using a list of bidirectional arcs:

In [ ]:
arcs = [(0,1), (0,3), (1,2), (1,4), (2,5), (4,3), (3,6), (4,7)]

From this, we derive our states and actions:

In [ ]:
states  = list(set([x for v in arcs for x in v]))
actions = states[:]
goal    = 7

transition_matrix = [
	[None if (i,j) not in arcs and (j,i) not in arcs else j for j in actions]
	for i in states
]

reward_matrix = [
	[None if (i,j) not in arcs and (j,i) not in arcs else 100 if j == goal else 0 for j in actions]
	for i in states
]

In [ ]:
def mprint(matrix):
  for row in matrix:
    print(' , '.join(
        ['{: >6}'.format(np.round(item, 1) if item is not None else ' ') for item in row]
    ))

In [ ]:
print("The transition matrix:")
mprint(transition_matrix)

The transition matrix:
       ,      1 ,        ,      3 ,        ,        ,        ,       
     0 ,        ,      2 ,        ,      4 ,        ,        ,       
       ,      1 ,        ,        ,        ,      5 ,        ,       
     0 ,        ,        ,        ,      4 ,        ,      6 ,       
       ,      1 ,        ,      3 ,        ,        ,        ,      7
       ,        ,      2 ,        ,        ,        ,        ,       
       ,        ,        ,      3 ,        ,        ,        ,       
       ,        ,        ,        ,      4 ,        ,        ,       


In [ ]:
print("The reward matrix:")
mprint(reward_matrix)

The reward matrix:
       ,      0 ,        ,      0 ,        ,        ,        ,       
     0 ,        ,      0 ,        ,      0 ,        ,        ,       
       ,      0 ,        ,        ,        ,      0 ,        ,       
     0 ,        ,        ,        ,      0 ,        ,      0 ,       
       ,      0 ,        ,      0 ,        ,        ,        ,    100
       ,        ,      0 ,        ,        ,        ,        ,       
       ,        ,        ,      0 ,        ,        ,        ,       
       ,        ,        ,        ,      0 ,        ,        ,       


First, we'll train an agent which does no exploration at all (`epsilon` = 0):

In [ ]:
Q = train_agent(transition_matrix, reward_matrix, end_states=[goal], gamma=0.8, alpha=1, epsilon=0)
print("\nThe trained Q matrix:")
mprint(Q)



The trained Q matrix:
   0.0 ,    0.0 ,    0.0 ,   64.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0
  51.2 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0
   0.0 ,   41.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0
   0.0 ,    0.0 ,    0.0 ,    0.0 ,   80.0 ,    0.0 ,    0.0 ,    0.0
   0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,  100.0
   0.0 ,    0.0 ,   32.8 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0
   0.0 ,    0.0 ,    0.0 ,   64.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0
   0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0


Now, we will use an exploration factor:

In [ ]:
Q = train_agent(transition_matrix, reward_matrix, end_states=[goal], gamma=0.8, alpha=1, epsilon=0.3)
print("\nThe trained Q matrix:")
mprint(Q)



The trained Q matrix:
   0.0 ,   64.0 ,    0.0 ,   64.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0
  51.2 ,    0.0 ,   51.2 ,    0.0 ,   80.0 ,    0.0 ,    0.0 ,    0.0
   0.0 ,   64.0 ,    0.0 ,    0.0 ,    0.0 ,   41.0 ,    0.0 ,    0.0
  51.2 ,    0.0 ,    0.0 ,    0.0 ,   80.0 ,    0.0 ,   51.2 ,    0.0
   0.0 ,   64.0 ,    0.0 ,   64.0 ,    0.0 ,    0.0 ,    0.0 ,  100.0
   0.0 ,    0.0 ,   51.2 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0
   0.0 ,    0.0 ,    0.0 ,   64.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0
   0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0 ,    0.0


Which differences do you see? In particular, take a look at the Q-values for the first row.